In [1]:
import numpy
import sys
import nltk 
nltk.download ('stopwords') 
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asada\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open("frnk123.txt", "rb") as file:
    binary_data = file.read() 

In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# Read the content of the file
with open("frnk123.txt", "rb") as file:
    content = file.read().decode('utf-8', errors='ignore')

def tokenize_words(input_text):
    input_text = input_text.lower()  # Convert text to lowercase
    tokenizer = RegexpTokenizer(r'\w+')  # Tokenize based on word boundaries
    tokens = tokenizer.tokenize(input_text)
    filtered = [token for token in tokens if token not in stopwords.words('english')]
    return " ".join(filtered)

processed_inputs = tokenize_words(content)
 


In [4]:
#char to numbers
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [5]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print("Total number of characters:", input_len)
print("Total vocab:",vocab_len)


Total number of characters: 255635
Total vocab: 43


In [6]:
seq_length = 100
x_data = []
y_data = [] 

In [7]:
#loop through the sequence
for i in range(0, input_len - seq_length):
    in_seq = processed_inputs[i:i + seq_length]
    out_seq = processed_inputs[i + seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print ("Total Patterns:", n_patterns) 
    

Total Patterns: 255535


In [8]:
#convert input sequence to np array and so on
x = numpy.reshape(x_data, (n_patterns, seq_length, 1))
x = x/float(vocab_len)

In [9]:
!pip install keras
from keras.utils import to_categorical


Defaulting to user installation because normal site-packages is not writeable


In [10]:
if len(y_data) == 0:
    print("y_data is empty.")
else:
    y = to_categorical(y_data)


In [11]:
import numpy as np

# Check data types and convert to integers if necessary
if not isinstance(y_data, (list, np.ndarray)):
    raise ValueError("y_data should be a list or numpy array.")
if y_data and not isinstance(y_data[0], int):
    y_data = [int(label) for label in y_data]

# Check data shape
if len(y_data) == 0:
    raise ValueError("y_data is empty.")
if len(set(y_data)) <= 0:
    raise ValueError("Invalid class labels in y_data.")

# Use to_categorical
y = to_categorical(y_data)


In [12]:
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax' ))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
filepath = 'model_weights_saved.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1,save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [15]:
model.fit(x,y, epochs=4, batch_size=256, callbacks=desired_callbacks)  

Epoch 1/4
999/999 [==============================] - ETA: 0s - loss: 2.8926
Epoch 1: loss improved from inf to 2.89262, saving model to model_weights_saved.hdf5
999/999 [==============================] - 4788s 5s/step - loss: 2.8926
Epoch 2/4


C:\Users\asada\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


999/999 [==============================] - ETA: 0s - loss: 2.5872
Epoch 2: loss improved from 2.89262 to 2.58725, saving model to model_weights_saved.hdf5
999/999 [==============================] - 4088s 4s/step - loss: 2.5872
Epoch 3/4
999/999 [==============================] - ETA: 0s - loss: 2.3950
Epoch 3: loss improved from 2.58725 to 2.39501, saving model to model_weights_saved.hdf5
999/999 [==============================] - 4082s 4s/step - loss: 2.3950
Epoch 4/4
999/999 [==============================] - ETA: 0s - loss: 2.2549
Epoch 4: loss improved from 2.39501 to 2.25495, saving model to model_weights_saved.hdf5
999/999 [==============================] - 4069s 4s/step - loss: 2.2549


In [16]:
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)


Shape of x: (255535, 100, 1)
Shape of y: (255535, 43)


In [17]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
#output of model back into characters
num_to_char = dict((i,c) for i,c in enumerate(chars))

In [19]:
#random seed to help generate
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" ffered late discovered unworthiness one beloved disposed set greater value tried worth show gratitud "


In [20]:
for i in range(1000):
    x = numpy.reshape(pattern, (1,len(pattern), 1))
    x = x/float(vocab_len)
    prediction = model.predict(x, verbose=0)   
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

d seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare se